In [3]:
# Import the libraries
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
#physical_devices = tf.config.list_physical_devices('GPU')
#tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)
import numpy as np
import matplotlib.pyplot as plt  
import pandas as pd
from numpy import loadtxt
from tensorflow.keras.models import load_model,save_model
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Activation,Flatten,Input
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import Adam,RMSprop
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import mean_squared_error
import math,json,os

#my scaler
import predict_utils
#read pre-trained scaler
from pickle import load

anomaly_threshold = 0.15
tolerance = 20

try:
    # Load the scaler
    scaler = load(open('scaler.pkl', 'rb'))
    # Load model
    predict_model = load_model("./conv_ae.h5")
except:
    print("loading error")

# Read Dataframe
data_root_path = "E:\Server_mantain\Spark_test\dataset\Zabbix_monitor_SB\concate_with_disk"
data5v_path = os.path.join(data_root_path,"predict_data5V.csv")
data6v_path = os.path.join(data_root_path,"predict_data6V.csv")
data5v = predict_utils.read_data(data5v_path,fill_zero=True)
data6v = predict_utils.read_data(data6v_path,fill_zero=True)

# Sliding window generate Input Dataframe
data_list = []
lag = 1
for i in range(0,data5v.shape[0]-lag,lag):
    data_list.append(data5v[i:i+100])
for i in range(0,data6v.shape[0]-lag,lag):
    data_list.append(data5v[i:i+100])

host                   0
CPU utilization (%)    0
Memory used  (%)       1
dtype: int64
host                   0
CPU utilization (%)    0
Memory used  (%)       0
dtype: int64
                                host  CPU utilization (%)  Memory used  (%)
dt                                                                         
2021-06-22 18:41:00  TWKHH01OPTIPW5V              19.8383            0.0000
2021-06-22 18:44:00  TWKHH01OPTIPW5V              18.7174           18.9454
2021-06-22 18:47:00  TWKHH01OPTIPW5V               1.8868           21.0194
2021-06-22 18:50:00  TWKHH01OPTIPW5V              11.0442           17.0870
2021-06-22 18:53:00  TWKHH01OPTIPW5V               2.0538           13.7514
host                   0
CPU utilization (%)    0
Memory used  (%)       0
dtype: int64
host                   0
CPU utilization (%)    0
Memory used  (%)       0
dtype: int64
                                host  CPU utilization (%)  Memory used  (%)
dt                                      

In [2]:
zip(data_list[0].values,data_list[0].index)

type(data_list[0].index)

pandas.core.indexes.datetimes.DatetimeIndex

In [4]:
from collections import Counter
In_data = data_list[0]
host = In_data.iloc[:,0].values

dict_keys(['TWKHH01OPTIPW5V'])

In [2]:
import random
len(data_list)

28797

In [17]:
num_list = list(range(len(data_list)))
random.shuffle(num_list) 
for sample_data in  random.sample(data_list,1) :

    
    In_data = sample_data
    
    usage = In_data.iloc[:,1:].values
    host = In_data.iloc[:,0].values
    date_time = In_data.index
    #print("{} ~ {}".format(In_data.index[0],In_data.index[-1]))
    # Fit scaler with shape (None,8)
    usage = np.concatenate([usage,usage,usage,usage],axis=1)
    usage = scaler.transform(usage)[:,:2]

    # Generate sliding windows
    window_size = 32

    X_list = []
    time_list = []
    for i in range(0,usage.shape[0]-window_size):
        time_list.append(date_time[i:i+window_size])
        X_list.append(usage[i:i+window_size,:])

    
    # X shape : (window_num,32,2)
    print(time_list[0][0],time_list[0][-1])

2021-07-11 16:56:00 2021-07-11 18:29:00


In [29]:
from scipy import stats
stats.mode(host)[0].item()

'TWKHH01OPTIPW5V'